# 🎯 PM OS - Product Manager Operating System

A multi-agent AI assistant for Product Managers.

**Run all cells in order (Shift+Enter)**

In [ ]:
#@title 1. Install Dependencies
!pip install streamlit anthropic pyngrok -q
print("✅ Dependencies installed!")

In [ ]:
#@title 2. Enter Your API Key
import os

OPENROUTER_API_KEY = "" #@param {type:"string"}
NGROK_AUTH_TOKEN = "" #@param {type:"string"}

os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

print("✅ API key set!" if OPENROUTER_API_KEY else "⚠️ Enter your OpenRouter API key above")
print("💡 Get one free at: https://openrouter.ai")

In [ ]:
#@title 3. Create PM OS Files
!mkdir -p pm_os/agents

# Create all the necessary files
files_content = {
    "pm_os/agents/__init__.py": '''from .framer import create_framer_agent
from .strategist import create_strategist_agent
from .aligner import create_aligner_agent
from .executor import create_executor_agent
from .narrator import create_narrator_agent
from .doc_engine import create_doc_engine_agent

FRAMER = create_framer_agent()
STRATEGIST = create_strategist_agent()
ALIGNER = create_aligner_agent()
EXECUTOR = create_executor_agent()
NARRATOR = create_narrator_agent()
DOC_ENGINE = create_doc_engine_agent()

AGENTS = {"framer": FRAMER, "strategist": STRATEGIST, "aligner": ALIGNER,
          "executor": EXECUTOR, "narrator": NARRATOR, "doc_engine": DOC_ENGINE}

def get_agent(name): return AGENTS.get(name.lower())
''',

    "pm_os/agents/base.py": '''import anthropic, json
from dataclasses import dataclass, field
from typing import Callable, Optional

@dataclass
class Tool:
    name: str
    description: str
    input_schema: dict
    function: Callable
    def to_anthropic_format(self): return {"name": self.name, "description": self.description, "input_schema": self.input_schema}

@dataclass
class AgentConfig:
    name: str
    emoji: str
    description: str
    system_prompt: str
    tools: list = field(default_factory=list)
    output_parser: Optional[Callable] = None

class BaseAgent:
    def __init__(self, config):
        self.config = config
        self.name = config.name
        self.emoji = config.emoji
        self.description = config.description
        self.system_prompt = config.system_prompt
        self.tools = config.tools
        self._tool_map = {t.name: t for t in self.tools}

    @property
    def display_name(self): return f"{self.emoji} {self.name}"

    def _get_client(self, api_key, provider):
        if provider == "anthropic": return anthropic.Anthropic(api_key=api_key)
        return anthropic.Anthropic(api_key=api_key, base_url="https://openrouter.ai/api/v1")

    def _get_model(self, provider): return "claude-sonnet-4-20250514" if provider == "anthropic" else "anthropic/claude-sonnet-4"

    def _execute_tool(self, name, tool_input):
        tool = self._tool_map.get(name)
        if not tool: return json.dumps({"error": f"Unknown tool: {name}"})
        try:
            result = tool.function(**tool_input)
            return result if isinstance(result, str) else json.dumps(result)
        except Exception as e: return json.dumps({"error": str(e)})

    def run(self, user_message, conversation_history=None, api_key=None, provider="openrouter", max_iterations=10):
        client, model = self._get_client(api_key, provider), self._get_model(provider)
        messages = list(conversation_history or []) + [{"role": "user", "content": user_message}]
        tools_api = [t.to_anthropic_format() for t in self.tools] if self.tools else None
        metadata = {"agent": self.name, "tools_used": [], "iterations": 0}
        while metadata["iterations"] < max_iterations:
            metadata["iterations"] += 1
            kwargs = {"model": model, "max_tokens": 4096, "system": self.system_prompt, "messages": messages}
            if tools_api: kwargs["tools"] = tools_api
            response = client.messages.create(**kwargs)
            if response.stop_reason == "tool_use":
                messages.append({"role": "assistant", "content": response.content})
                tool_results = []
                for block in response.content:
                    if block.type == "tool_use":
                        result = self._execute_tool(block.name, block.input)
                        metadata["tools_used"].append({"name": block.name, "input": block.input, "output": result})
                        tool_results.append({"type": "tool_result", "tool_use_id": block.id, "content": result})
                messages.append({"role": "user", "content": tool_results})
            else:
                return "".join(b.text for b in response.content if hasattr(b, "text")), metadata
        return "Max iterations.", metadata

def extract_section(text, name):
    if not text: return ""
    for p in [f"**{name.lower()}", f"## {name.lower()}", f"### {name.lower()}"]:
        i = text.lower().find(p)
        if i != -1:
            end = len(text)
            for m in ["\\n## ", "\\n### ", "\\n**"]: 
                j = text.find(m, i+10)
                if j != -1 and j < end: end = j
            return text[i:end].strip()
    return ""
''',

    "pm_os/agents/framer.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

def log_why(why_number, question, answer): return json.dumps({"why": why_number, "q": question, "a": answer})
def generate_problem_statement(user_type, need, insight): return json.dumps({"statement": f"{user_type} needs {need} because {insight}"})
def suggest_next_steps(root_cause, context=""): return json.dumps({"root_cause": root_cause})

TOOLS = [
    Tool("log_why", "Log a Why question and answer in the 5 Whys analysis", {"type":"object","properties":{"why_number":{"type":"integer","description":"The Why number (1-5)"},"question":{"type":"string","description":"The Why question being asked"},"answer":{"type":"string","description":"The answer to this Why"}},"required":["why_number","question","answer"]}, log_why),
    Tool("generate_problem_statement", "Generate the final problem statement in format: [User] needs [need] because [insight]", {"type":"object","properties":{"user_type":{"type":"string","description":"The user or customer type"},"need":{"type":"string","description":"What the user needs"},"insight":{"type":"string","description":"The insight or reason why"}},"required":["user_type","need","insight"]}, generate_problem_statement),
    Tool("suggest_next_steps", "Record root cause and prepare to suggest next steps", {"type":"object","properties":{"root_cause":{"type":"string","description":"The identified root cause from 5 Whys"},"context":{"type":"string","description":"Additional context"}},"required":["root_cause"]}, suggest_next_steps),
]

PROMPT = """You are the Framer Agent, an expert at problem definition using the 5 Whys technique.

Your process:
1. Acknowledge the surface problem the user described
2. Run 5 Whys analysis - use log_why tool for EACH why (1-5)
3. Generate a problem statement using generate_problem_statement
4. Use suggest_next_steps with the root cause
5. Provide 3-5 actionable recommendations

Output format:
## Problem Analysis
**Surface Problem:** [what user described]
**5 Whys Deep Dive:**
1. Why? → [answer]
2. Why? → [answer]
3. Why? → [answer]
4. Why? → [answer]
5. Why? → [ROOT CAUSE]
**Root Cause Identified:** [clear statement]
**Problem Statement:** > [user] needs [need] because [insight]
**Recommended Next Steps:**
- [action 1]
- [action 2]
- [action 3]"""

def create_framer_agent(): return BaseAgent(AgentConfig("Framer", "🔍", "Problem definition using 5 Whys - finds root causes", PROMPT, TOOLS))
''',

    "pm_os/agents/strategist.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

_opts = []
def add_option(name, description): _opts.append({"name":name,"desc":description}); return json.dumps({"added":name})
def score_option(name, impact, effort, confidence): 
    score = round((impact * confidence) / max(effort, 1), 2)
    return json.dumps({"name":name,"impact":impact,"effort":effort,"confidence":confidence,"score":score})
def compare_options(): return json.dumps({"options":_opts})
def analyze_tradeoffs(option_a, option_b, key_difference): return json.dumps({"a":option_a,"b":option_b,"tradeoff":key_difference})

TOOLS = [
    Tool("add_option", "Add an option to evaluate", {"type":"object","properties":{"name":{"type":"string","description":"Option name"},"description":{"type":"string","description":"What this option entails"}},"required":["name","description"]}, add_option),
    Tool("score_option", "Score an option (1-5 scale)", {"type":"object","properties":{"name":{"type":"string","description":"Option name"},"impact":{"type":"integer","description":"Business impact (1-5)"},"effort":{"type":"integer","description":"Implementation effort (1-5, 1=easy)"},"confidence":{"type":"integer","description":"Execution confidence (1-5)"}},"required":["name","impact","effort","confidence"]}, score_option),
    Tool("compare_options", "Compare all scored options", {"type":"object","properties":{}}, compare_options),
    Tool("analyze_tradeoffs", "Analyze tradeoffs between two options", {"type":"object","properties":{"option_a":{"type":"string"},"option_b":{"type":"string"},"key_difference":{"type":"string"}},"required":["option_a","option_b","key_difference"]}, analyze_tradeoffs),
]

PROMPT = """You are the Strategist Agent, expert at prioritization using scoring frameworks.

Your process:
1. Identify all options using add_option
2. Score each option using score_option (impact, effort, confidence 1-5)
3. Use compare_options to see all options
4. Use analyze_tradeoffs for key decisions
5. Make a clear recommendation

Output format:
## Prioritization Analysis
**Options:**
1. [Option A] - [description]
2. [Option B] - [description]

**Scoring Matrix:**
| Option | Impact | Effort | Confidence | Score |
|--------|--------|--------|------------|-------|
| A      | X      | X      | X          | X.XX  |

**Key Trade-offs:** [analysis]
**Recommendation:** [CLEAR CHOICE with reasoning]
**Next Steps:** [immediate actions]"""

def create_strategist_agent(): return BaseAgent(AgentConfig("Strategist", "📊", "Prioritization with scoring frameworks", PROMPT, TOOLS))
''',

    "pm_os/agents/aligner.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

def add_stakeholder(name, role, interest, influence): return json.dumps({"stakeholder":name,"role":role})
def define_ask(what, why, by_when): return json.dumps({"ask":what,"why":why,"deadline":by_when})
def prepare_objection_response(objection, response): return json.dumps({"objection":objection,"response":response})
def create_talking_point(point, supporting_data): return json.dumps({"point":point,"data":supporting_data})

TOOLS = [
    Tool("add_stakeholder", "Map a stakeholder", {"type":"object","properties":{"name":{"type":"string"},"role":{"type":"string"},"interest":{"type":"string"},"influence":{"type":"string"}},"required":["name","role","interest","influence"]}, add_stakeholder),
    Tool("define_ask", "Define what you're asking for", {"type":"object","properties":{"what":{"type":"string"},"why":{"type":"string"},"by_when":{"type":"string"}},"required":["what","why","by_when"]}, define_ask),
    Tool("prepare_objection_response", "Prepare response to potential objection", {"type":"object","properties":{"objection":{"type":"string"},"response":{"type":"string"}},"required":["objection","response"]}, prepare_objection_response),
    Tool("create_talking_point", "Create a key talking point", {"type":"object","properties":{"point":{"type":"string"},"supporting_data":{"type":"string"}},"required":["point","supporting_data"]}, create_talking_point),
]

PROMPT = """You are the Aligner Agent, expert at stakeholder alignment and executive communication.

Your process:
1. Map stakeholders using add_stakeholder (name, role, interest, influence level)
2. Define the ask using define_ask (what, why, deadline)
3. Prepare objection responses using prepare_objection_response
4. Create talking points using create_talking_point

Output format:
## Alignment Strategy
**Stakeholder Map:**
| Name | Role | Interest | Influence |
**The Ask:** [what you need, why, by when]
**Talking Points:** [key messages with data]
**Objection Handling:** [anticipated pushback and responses]"""

def create_aligner_agent(): return BaseAgent(AgentConfig("Aligner", "🤝", "Stakeholder alignment and executive communication", PROMPT, TOOLS))
''',

    "pm_os/agents/executor.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

def add_feature(name, description, user_value): return json.dumps({"feature":name,"value":user_value})
def classify_feature(name, classification, reason): return json.dumps({"feature":name,"class":classification,"reason":reason})
def define_mvp(features, rationale): return json.dumps({"mvp":features,"rationale":rationale})
def add_checklist_item(item, owner): return json.dumps({"item":item,"owner":owner})
def set_launch_criteria(metric, target): return json.dumps({"metric":metric,"target":target})

TOOLS = [
    Tool("add_feature", "Add a feature to evaluate", {"type":"object","properties":{"name":{"type":"string"},"description":{"type":"string"},"user_value":{"type":"string"}},"required":["name","description","user_value"]}, add_feature),
    Tool("classify_feature", "Classify as must-have, nice-to-have, or cut", {"type":"object","properties":{"name":{"type":"string"},"classification":{"type":"string","enum":["must-have","nice-to-have","cut"]},"reason":{"type":"string"}},"required":["name","classification","reason"]}, classify_feature),
    Tool("define_mvp", "Define the MVP scope", {"type":"object","properties":{"features":{"type":"array","items":{"type":"string"}},"rationale":{"type":"string"}},"required":["features","rationale"]}, define_mvp),
    Tool("add_checklist_item", "Add launch checklist item", {"type":"object","properties":{"item":{"type":"string"},"owner":{"type":"string"}},"required":["item","owner"]}, add_checklist_item),
    Tool("set_launch_criteria", "Set success metric", {"type":"object","properties":{"metric":{"type":"string"},"target":{"type":"string"}},"required":["metric","target"]}, set_launch_criteria),
]

PROMPT = """You are the Executor Agent, expert at MVP scoping and shipping products.

Your process:
1. List all features using add_feature
2. Classify each using classify_feature (must-have/nice-to-have/cut)
3. Define MVP using define_mvp
4. Add launch checklist items
5. Set launch criteria/success metrics

Output format:
## MVP Scope
**Features Evaluated:**
| Feature | Classification | Reason |
**MVP Definition:** [included features and rationale]
**Cut List:** [features cut and why]
**Launch Checklist:** [items with owners]
**Success Criteria:** [metrics and targets]"""

def create_executor_agent(): return BaseAgent(AgentConfig("Executor", "🚀", "MVP scoping and shipping", PROMPT, TOOLS))
''',

    "pm_os/agents/narrator.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

def draft_tldr(summary): return json.dumps({"tldr":summary})
def structure_what(content): return json.dumps({"what":content})
def structure_why(content): return json.dumps({"why":content})
def structure_ask(content): return json.dumps({"ask":content})
def add_supporting_data(metric, value): return json.dumps({"metric":metric,"value":value})
def flag_risk(risk, mitigation): return json.dumps({"risk":risk,"mitigation":mitigation})

TOOLS = [
    Tool("draft_tldr", "Draft the TL;DR summary", {"type":"object","properties":{"summary":{"type":"string"}},"required":["summary"]}, draft_tldr),
    Tool("structure_what", "Structure the What section", {"type":"object","properties":{"content":{"type":"string"}},"required":["content"]}, structure_what),
    Tool("structure_why", "Structure the Why section", {"type":"object","properties":{"content":{"type":"string"}},"required":["content"]}, structure_why),
    Tool("structure_ask", "Structure the Ask section", {"type":"object","properties":{"content":{"type":"string"}},"required":["content"]}, structure_ask),
    Tool("add_supporting_data", "Add supporting data point", {"type":"object","properties":{"metric":{"type":"string"},"value":{"type":"string"}},"required":["metric","value"]}, add_supporting_data),
    Tool("flag_risk", "Flag a risk with mitigation", {"type":"object","properties":{"risk":{"type":"string"},"mitigation":{"type":"string"}},"required":["risk","mitigation"]}, flag_risk),
]

PROMPT = """You are the Narrator Agent, expert at executive summaries and stakeholder updates.

Your process:
1. Draft TL;DR using draft_tldr (1-2 sentences)
2. Structure What using structure_what
3. Structure Why using structure_why  
4. Structure Ask using structure_ask
5. Add supporting data points
6. Flag any risks with mitigations

Output format:
## Executive Summary
**TL;DR:** [1-2 sentence summary]
**What:** [what's happening/proposed]
**Why:** [why it matters]
**Ask:** [what you need from the reader]
**Key Data:** [supporting metrics]
**Risks:** [risks and mitigations]"""

def create_narrator_agent(): return BaseAgent(AgentConfig("Narrator", "📝", "Executive summaries and updates", PROMPT, TOOLS))
''',

    "pm_os/agents/doc_engine.py": '''from .base import BaseAgent, AgentConfig, Tool
import json

def set_document_metadata(title, author, date): return json.dumps({"title":title,"author":author,"date":date})
def define_problem(statement, impact): return json.dumps({"problem":statement,"impact":impact})
def add_goal(goal, metric, target): return json.dumps({"goal":goal,"metric":metric,"target":target})
def add_user_story(persona, action, benefit): return json.dumps({"story":f"As {persona}, I want {action} so that {benefit}"})
def add_requirement(req_id, description, priority): return json.dumps({"id":req_id,"desc":description,"priority":priority})
def define_scope(in_scope, out_scope): return json.dumps({"in":in_scope,"out":out_scope})
def add_timeline_phase(phase, duration, deliverables): return json.dumps({"phase":phase,"duration":duration,"deliverables":deliverables})
def add_open_question(question, owner): return json.dumps({"question":question,"owner":owner})

TOOLS = [
    Tool("set_document_metadata", "Set PRD metadata", {"type":"object","properties":{"title":{"type":"string"},"author":{"type":"string"},"date":{"type":"string"}},"required":["title","author","date"]}, set_document_metadata),
    Tool("define_problem", "Define the problem statement", {"type":"object","properties":{"statement":{"type":"string"},"impact":{"type":"string"}},"required":["statement","impact"]}, define_problem),
    Tool("add_goal", "Add a goal with success metric", {"type":"object","properties":{"goal":{"type":"string"},"metric":{"type":"string"},"target":{"type":"string"}},"required":["goal","metric","target"]}, add_goal),
    Tool("add_user_story", "Add a user story", {"type":"object","properties":{"persona":{"type":"string"},"action":{"type":"string"},"benefit":{"type":"string"}},"required":["persona","action","benefit"]}, add_user_story),
    Tool("add_requirement", "Add a requirement", {"type":"object","properties":{"req_id":{"type":"string"},"description":{"type":"string"},"priority":{"type":"string"}},"required":["req_id","description","priority"]}, add_requirement),
    Tool("define_scope", "Define in/out of scope", {"type":"object","properties":{"in_scope":{"type":"array","items":{"type":"string"}},"out_scope":{"type":"array","items":{"type":"string"}}},"required":["in_scope","out_scope"]}, define_scope),
    Tool("add_timeline_phase", "Add timeline phase", {"type":"object","properties":{"phase":{"type":"string"},"duration":{"type":"string"},"deliverables":{"type":"string"}},"required":["phase","duration","deliverables"]}, add_timeline_phase),
    Tool("add_open_question", "Add open question", {"type":"object","properties":{"question":{"type":"string"},"owner":{"type":"string"}},"required":["question","owner"]}, add_open_question),
]

PROMPT = """You are the Doc Engine Agent, expert at creating Product Requirements Documents.

Your process:
1. Set metadata using set_document_metadata
2. Define problem using define_problem
3. Add goals using add_goal (with metrics)
4. Add user stories using add_user_story
5. Add requirements using add_requirement
6. Define scope using define_scope
7. Add timeline phases
8. Add open questions

Output format: A complete PRD with all sections filled out."""

def create_doc_engine_agent(): return BaseAgent(AgentConfig("Doc Engine", "📄", "PRD and document generation", PROMPT, TOOLS))
''',

    "pm_os/router.py": '''import anthropic
from agents import AGENTS, get_agent

PROMPT = """Classify the user's intent and return ONLY the agent name (lowercase, no explanation):
- framer: vague problems, root cause analysis, "why" questions
- strategist: prioritization, decisions, trade-offs, comparisons
- aligner: stakeholders, meetings, executives, alignment
- executor: MVP, shipping, launch, features to cut
- narrator: summaries, updates, release notes
- doc_engine: PRDs, specs, documentation"""

def route_message(msg, api_key, provider="openrouter"):
    client = anthropic.Anthropic(api_key=api_key, base_url="https://openrouter.ai/api/v1") if provider=="openrouter" else anthropic.Anthropic(api_key=api_key)
    model = "anthropic/claude-sonnet-4" if provider=="openrouter" else "claude-sonnet-4-20250514"
    r = client.messages.create(model=model, max_tokens=50, system=PROMPT, messages=[{"role":"user","content":msg}])
    name = r.content[0].text.strip().lower()
    for n in AGENTS: 
        if n in name: return n, get_agent(n)
    return "framer", get_agent("framer")
''',

    "pm_os/memory.py": '''import json
from dataclasses import dataclass, field, asdict
from datetime import datetime
from typing import Optional

@dataclass
class Decision:
    timestamp: str
    agent_name: str
    agent_emoji: str
    user_query: str
    decision_summary: str
    context: Optional[str] = None

@dataclass
class ConversationTurn:
    timestamp: str
    user_message: str
    agent_name: str
    agent_response: str

@dataclass
class SessionMemory:
    session_id: str
    created_at: str
    conversation: list = field(default_factory=list)
    decisions: list = field(default_factory=list)

    def add_turn(self, msg, agent, response):
        self.conversation.append(ConversationTurn(datetime.now().isoformat(), msg, agent, response))

    def add_decision(self, agent_name, agent_emoji, query, summary, context=None):
        self.decisions.append(Decision(datetime.now().isoformat(), agent_name, agent_emoji, query, summary, context))

    def get_decisions_markdown(self):
        if not self.decisions: return "*No decisions yet*"
        return "\\n".join([f"### {i+1}. {d.agent_emoji} {d.agent_name}\\n**Query:** {d.user_query[:80]}...\\n**Decision:** {d.decision_summary}\\n" for i,d in enumerate(self.decisions)])

def create_session(): return SessionMemory(datetime.now().strftime("%Y%m%d_%H%M%S"), datetime.now().isoformat())

def extract_decision_summary(agent, response):
    for m in ["**recommendation:**","**problem statement:**","**tl;dr:**","**root cause"]:
        if m in response.lower():
            i = response.lower().find(m)
            return response[i:i+200].strip()
    return None
''',

    "pm_os/evaluation.py": '''from dataclasses import dataclass
from datetime import datetime

@dataclass
class Evaluation:
    agent_name: str
    completeness: int
    user_rating: str = None

class EvaluationStore:
    def __init__(self): self.evals = []
    def add_evaluation(self, agent, query, response, tools):
        self.evals.append(Evaluation(agent, min(5, 2 + response.count("##"))))
    def rate_last(self, rating):
        if self.evals: self.evals[-1].user_rating = rating
    def get_stats_markdown(self):
        if not self.evals: return "*No evaluations yet*"
        return f"**Total:** {len(self.evals)} | 👍 {sum(1 for e in self.evals if e.user_rating=='up')} | 👎 {sum(1 for e in self.evals if e.user_rating=='down')}"

_store = None
def get_evaluation_store():
    global _store
    if _store is None: _store = EvaluationStore()
    return _store
def reset_evaluation_store():
    global _store
    _store = EvaluationStore()
''',

    "pm_os/web_search.py": '''_key = None
def set_serpapi_key(k): global _key; _key = k
''',

    "pm_os/sheets_export.py": '''import csv, io
_id = None
def set_sheet_id(i): global _id; _id = i
def get_sheet_url(): return f"https://docs.google.com/spreadsheets/d/{_id}/edit" if _id else None
def generate_decisions_csv(decs):
    o = io.StringIO()
    w = csv.writer(o)
    w.writerow(["Date","Agent","Query","Decision"])
    for d in decs: w.writerow([d.get("timestamp","")[:10], d.get("agent_name",""), d.get("user_query","")[:100], d.get("decision_summary","")[:200]])
    return o.getvalue()
''',

    "pm_os/app.py": '''import os
import streamlit as st
from agents import AGENTS, get_agent
from router import route_message
from memory import create_session, extract_decision_summary
from evaluation import get_evaluation_store, reset_evaluation_store
from sheets_export import generate_decisions_csv

st.set_page_config(page_title="PM OS", page_icon="🎯", layout="wide")

if "messages" not in st.session_state: st.session_state.messages = []
if "memory" not in st.session_state: st.session_state.memory = create_session()
if "evals" not in st.session_state: st.session_state.evals = get_evaluation_store()

def process(msg, key):
    if not msg.strip() or not key.strip(): 
        st.error("Please enter your OpenRouter API key in the sidebar")
        return
    try:
        name, agent = route_message(msg, key, "openrouter")
        hist = [{"role":m["role"],"content":m["content"]} for m in st.session_state.messages]
        resp, meta = agent.run(msg, hist, key, "openrouter")
        tools = [t["name"] for t in meta.get("tools_used",[])]
        st.session_state.evals.add_evaluation(name, msg, resp, tools)
        out = f"### {agent.emoji} {agent.name} Agent\\n*{agent.description}*\\n\\n---\\n\\n{resp}"
        if tools: out += f"\\n\\n---\\n*Tools used: {', '.join(tools)}*"
        st.session_state.messages.append({"role":"assistant","content":out})
        st.session_state.memory.add_turn(msg, name, resp)
        dec = extract_decision_summary(name, resp)
        if dec: st.session_state.memory.add_decision(agent.name, agent.emoji, msg, dec)
    except Exception as e:
        st.session_state.messages.append({"role":"assistant","content":f"**Error:** {str(e)}"})

def clear():
    st.session_state.messages = []
    st.session_state.memory = create_session()
    reset_evaluation_store()
    st.session_state.evals = get_evaluation_store()

st.title("🎯 PM OS")
st.markdown("*Your AI Product Management Operating System*")

with st.sidebar:
    st.header("⚙️ Settings")
    key = st.text_input("OpenRouter API Key", type="password", value=os.environ.get("OPENROUTER_API_KEY",""), help="Get one at openrouter.ai")
    st.divider()
    if st.button("🗑️ Clear Chat", use_container_width=True): clear(); st.rerun()
    st.divider()
    st.markdown("**Available Agents:**")
    for n,a in AGENTS.items():
        st.markdown(f"{a.emoji} **{a.name}**")

t1, t2, t3 = st.tabs(["💬 Chat", "📋 Decisions", "🤖 Agents"])

with t1:
    for m in st.session_state.messages:
        with st.chat_message(m["role"]): st.markdown(m["content"])
    c1,c2 = st.columns(2)
    with c1:
        if st.button("👍 Helpful"): st.session_state.evals.rate_last("up"); st.toast("Thanks!")
    with c2:
        if st.button("👎 Not helpful"): st.session_state.evals.rate_last("down"); st.toast("Feedback noted")
    if p := st.chat_input("Ask a PM question..."):
        st.session_state.messages.append({"role":"user","content":p})
        with st.chat_message("user"): st.markdown(p)
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                process(p, key)
            if st.session_state.messages:
                st.markdown(st.session_state.messages[-1]["content"])

with t2:
    st.subheader("📋 Decision Log")
    if st.session_state.memory.decisions:
        data = [{"agent_name":d.agent_name,"agent_emoji":d.agent_emoji,"user_query":d.user_query,"decision_summary":d.decision_summary,"timestamp":d.timestamp} for d in st.session_state.memory.decisions]
        st.download_button("📥 Export CSV", generate_decisions_csv(data), "decisions.csv", "text/csv")
    st.markdown(st.session_state.memory.get_decisions_markdown())

with t3:
    st.subheader("🤖 Available Agents")
    for n,a in AGENTS.items():
        with st.expander(f"{a.emoji} {a.name} Agent"):
            st.write(f"**Description:** {a.description}")
            st.write(f"**Tools:** {len(a.tools)}")
            for t in a.tools:
                st.markdown(f"- `{t.name}`: {t.description[:60]}...")

st.divider()
st.caption("PM OS v2.5 | Built with Streamlit + Claude")
'''
}

for path, content in files_content.items():
    with open(path, "w") as f:
        f.write(content)
    print(f"✅ Created {path}")

print("\\n✅ All PM OS files created!")

In [ ]:
#@title 4. Launch PM OS
import subprocess
import time

# Start Streamlit
subprocess.Popen(["streamlit", "run", "pm_os/app.py", "--server.port", "8501", "--server.headless", "true"])
time.sleep(5)
print("✅ Streamlit started!")

# Create public URL
from pyngrok import ngrok
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

url = ngrok.connect(8501)
print(f"\n🎯 PM OS is running at: {url}")
print("\n👆 Click the link above to open PM OS!")

## Done! 🎉

Click the ngrok URL above to open PM OS.

**Example prompts to try:**
- "Should we prioritize AI features or enterprise security?"
- "Users are signing up but not completing onboarding"
- "Write a PRD for a new onboarding flow"
- "I have a meeting with my CEO tomorrow about Q1 priorities"
- "Help me cut this feature list to an MVP"